In [1]:
import numpy as np
import pickle
import tensorflow as tf
import sys

if sys.platform == "win32":
    sys.path.append(r"C:\Users\vik\Dropbox\Code\Python\structural_engineering")
else:
    sys.path.append("/home/ritchie46/Dropbox/Code/Python/structural_engineering")

from anastruct.fem.system import SystemElements
import matplotlib.pyplot as plt
%matplotlib inline
import math

In [2]:

class Environment:
    def __init__(self, length=3, height=2, optimize='moment'):
        self.length = length
        self.height = height
        self.state = None
        self.n = None
        self.actions_chosen = None
        self.no_action = None
        self.action_space = {0, 1, 2, 3, 4, 5, 6, 7}
        self.valid_actions = None
        self.optimize = optimize
        self.result_map = {}
        self.ss = None
        self.current_distance = length - 1

        # actions
        right = 0
        left = 4
        up = 2
        down = 6
        up_right = 1
        up_left = 3
        down_right = 7
        down_left = 5
        
        # If the state is a flattened array. This maps to the index displacements.
        self.move_map = {right: 1,
                         left: -1,
                         up: -length,
                         down: length,
                         up_right: -length + 1,
                         up_left: -length - 1,
                         down_right: length + 1,
                         down_left: length -1}
            
    def reset(self):
        self.state = np.zeros((self.height, self.length))
        self.n = 1
        self.actions_chosen = 0
        self.state[-1][0] = self.n
        self.det_valid_actions()
        return self.state.ravel()
    
    def return_action(self, r):
        done = False
        self.det_valid_actions()
        s = np.array(self.state.ravel())
        i = np.where(s == self.n)
        s[s > 0] = -1
        s[i] = 1
                
        # Bridge is build
        if self.state[-1][-1] != 0:
            r = r + 2 + 1 / (self.structure() / (1 / 16 * self.length**2))**2 # that is moment to the power 2
            done = True
            return s, r, done
        
        if len(self.valid_actions) == 0:
            done = True
            r -= 2
        
        return s, r, done
    
    def det_valid_actions(self):
        no_action = set()
        right = 0
        left = 4
        top = 2
        down = 6
        top_right = 1
        top_left = 3
        down_right = 7
        down_left = 5
        
        # current location
        row, col = np.where(self.state == self.n)
                
        # right:
        try:
            if self.state[row, col + 1] != 0:
                no_action.add(right)
        except IndexError:
            no_action.add(right)
 
        if col - 1 < 0:
            no_action.add(left)
        elif self.state[row, col - 1] != 0:
            no_action.add(left)

        if row - 1 < 0:
            no_action.add(top)
        elif self.state[row - 1, col] != 0:
            no_action.add(top)
            
        try:
            if self.state[row + 1, col] != 0:
                no_action.add(down)
        except IndexError:
            no_action.add(down)
            
        if col -1 < 0 or row + 1 == self.height:
            no_action.add(down_left)
        elif self.state[row + 1, col - 1] != 0:
            no_action.add(down_left)

        try:
            if self.state[row + 1, col + 1] != 0:
                no_action.add(down_right)
        except IndexError:
            no_action.add(down_right)
            
        if row - 1 < 0 or col - 1 < 0:
            no_action.add(top_left)
        elif self.state[row -1, col - 1] != 0:
            no_action.add(top_left)

        if row - 1 < 0 or col + 1 == self.length:
            no_action.add(top_right)
        elif self.state[row - 1, col + 1] != 0:
                no_action.add(top_right)
            
        self.no_action = no_action
        self.valid_actions = list(self.action_space - no_action)
        
    
    def step(self, a):
        """
        :param a: (int) action direction
        
        → 0
        ↗ 1
        ↑ 2
        ↖ 3
        ← 4
        ↙ 5
        ↓ 6
        ↘ 7
        """
        self.actions_chosen += 1

        flat_location_index = np.argwhere(self.state.ravel() == self.n)
                                
        if a in self.no_action:
            return self.return_action(-0.2)
        
        # there is a valid action, make a move
        self.n += 1
        move = self.move_map[a]
        self.state.ravel()[flat_location_index + move] = self.n
        
        row, col = np.where(self.state == self.n)
        y = self.height - 1 - row[0]
        x = self.length - 1  - col[0]
        
#         distance = (x**2 + y**2)**0.5
#         d_distance = self.current_distance - distance
#         self.current_distance = distance

        return self.return_action(-0.1)
    
    def structure(self):
        self.ss = SystemElements()
        last_loc = [0, 0]
        for i in range(2, self.n + 1):
            row, col = np.where(i  == self.state)
            
            y = self.height - 1 - row[0]
            x = col[0] 

            current_loc = [x, y]
            self.ss.add_element([last_loc, [x, y]], g=-1)
            last_loc = current_loc
        
#         n_nodes = len(self.ss.node_map)
#         forces = -1 / (n_nodes - 2)
#         for i in range(2, n_nodes):
#             self.ss.point_load(node_id=i, Fz=forces)
  
        self.ss.add_support_hinged(1)
        self.ss.add_support_hinged(len(self.ss.node_map))
        self.ss.solve()
        
        f_max = np.max(np.abs(self.ss.get_element_result_range(self.optimize)))
        
        return f_max
    
    def sample_action(self):
        return np.random.randint(0, 8) #self.valid_actions[np.random.randint(0, len(self.valid_actions))]

    

        
def test_env():
    env = Environment(4, 4)
    s = env.reset()

    print(env.step(0))
    print(env.state)
    print(env.step(2))
    print(env.state)
    print(env.step(1))
    print(env.state)
    print(env.step(7))
    print(env.state)
    print(env.step(6))
    print(env.state)

#     env.step(0)
#     print(env.state, "\n")
#     env.step(7)
#     print(env.state)
#     env.structure()
    
test_env()

(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,
        1.,  0.,  0.]), -0.1, False)
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 1.  2.  0.  0.]]
(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -1.,
       -1.,  0.,  0.]), -0.1, False)
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  3.  0.  0.]
 [ 1.  2.  0.  0.]]
(array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -1.,  0.,  0., -1.,
       -1.,  0.,  0.]), -0.1, False)
[[ 0.  0.  0.  0.]
 [ 0.  0.  4.  0.]
 [ 0.  3.  0.  0.]
 [ 1.  2.  0.  0.]]
(array([ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -1.,  0.,  1., -1.,
       -1.,  0.,  0.]), -0.1, False)
[[ 0.  0.  0.  0.]
 [ 0.  0.  4.  0.]
 [ 0.  3.  0.  5.]
 [ 1.  2.  0.  0.]]
(array([ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -1.,  0., -1., -1.,
       -1.,  0.,  1.]), 3.4441558772842846, True)
[[ 0.  0.  0.  0.]
 [ 0.  0.  4.  0.]
 [ 0.  3.  0.  5.]
 [ 1.  2.  0.  6.]]


In [3]:
# https://theneuralperspective.com/2016/11/25/reinforcement-learning-rl-policy-gradients-i/

def leaky_relu(x, alpha=0.1):
    return tf.maximum(x, alpha * x)

class Agent:
    def __init__(self, data_size, hidden_size, action_space, learning_rate, name):
        """
        :param data_size: (int) Columns of the data vector.
        :param hidden_size: (int) No. of hidden nodes.
        :param action_space: (int) No. of outputs.
        :param learning_rate: (flt)
        """
      
        if len(hidden_size) == 1:
            one_hidden_layer = True
            hidden_size.append(hidden_size[0])
        else:
            one_hidden_layer = False

        
        # Step 1: Feed forward
        # The argmax is the maximum Q-value.
        self.input_s = tf.placeholder(tf.float32, [None, data_size], name="input_s")
        self.w1 = tf.get_variable(f"{name}_w1", shape=[data_size, hidden_size[0]], initializer=tf.contrib.layers.xavier_initializer())
        self.b1 = tf.get_variable(f"{name}_b1", shape=(hidden_size[0], ), initializer=tf.zeros_initializer())
        self.layer_1 = leaky_relu(tf.matmul(self.input_s, self.w1) + self.b1)
        
        self.w2 = tf.get_variable(f"{name}_w2", shape=[hidden_size[0], hidden_size[1]], initializer=tf.contrib.layers.xavier_initializer())
        self.b2 = tf.get_variable(f"{name}_b2", shape=(hidden_size[1], ), initializer=tf.zeros_initializer())
        self.layer_2 = leaky_relu(tf.matmul(self.layer_1, self.w2) + self.b2)
                     
        self.w_out = tf.get_variable(f"{name}_w_out", shape=[hidden_size[1], action_space], initializer=tf.contrib.layers.xavier_initializer())
        self.b_out = tf.get_variable(f"{name}_b_out", shape=(action_space, ), initializer=tf.zeros_initializer())
        
        if one_hidden_layer:
            hidden_layer = self.layer_1
        else:
            hidden_layer = self.layer_2
        
        # argmax(Q(s, a)) 
        self.predict_Q = tf.matmul(hidden_layer, self.w_out) + self.b_out # actual Q-value
        self.p = tf.nn.softmax(self.predict_Q)
        self.Q_a = tf.argmax(self.predict_Q, 1)
        self.saver = tf.train.Saver()
        
        # Step 2: Determine loss / gradients. 
        # One hot encoded actions
        self.executed_actions = tf.placeholder(tf.int32)
        
        self.one_hot = tf.one_hot(self.executed_actions, 8)
        self.Q = tf.reduce_sum(tf.multiply(self.predict_Q, self.one_hot), axis=1)
        self.next_Q_r = tf.placeholder(tf.float32)

         # Loss
         # mse: (     target      -    prediction)^2
         #      r + max(Q(s', a') -    Q(s, a) )^2
        
        #self.loss = tf.losses.huber_loss(self.next_Q_r, - self.Q, delta=15)
#         self.clipped_error = tf.maximum(tf.abs(self.next_Q_r - self.Q), tf.ones(tf.shape(self.Q)))
#         self.loss = tf.reduce_sum(tf.square(self.clipped_error))       

        starter_learning_rate = learning_rate
        self.train_count = tf.Variable(0, trainable=False, name=f"{name}_train_count")
        learning_rate = tf.train.exponential_decay(starter_learning_rate, self.train_count,
                                                   1000, 0.96)
    
        self.loss = tf.reduce_sum(tf.square(self.next_Q_r - self.Q))   
        optimizer = tf.train.AdamOptimizer(learning_rate)
        
        self.train = optimizer.minimize(self.loss, self.train_count)
        
        
        
        

In [4]:
def discounted_reward(r, gamma):
    """
    The reward for a given state. Is the reward for that state + the discounted sum of future rewards.
    
    :param r: (array) Rewards.
    :param gamma: (flt) Discount factor
    """
    return np.cumsum(r * gamma**(np.arange(len(r)))[::-1])[::-1]

def prepare_update_target(trainables, tau=0.05):
    """
    The weights and biases of the target will be a depended of the primary network.
    
    wb[target] = tau * wb[primary] + (1-tau) * wb[target]
    
    This is a tensorflow operation and still needs to be run with Session.run(operation_holder)
    
    :trainables (tf.trainable_variables)
    :tau (flt) Rate to update the target graph
    """
    
    operation_holder = []
    n_variables = len(trainables) // 2 # the agent has half and the target has half
    
    for i, v in enumerate(trainables[0: n_variables]):
        operation_holder.append(trainables[i + n_variables].assign(v.value() * tau) +
                                ((1 - tau) * trainables[i + n_variables].value()))
    return operation_holder

def update_target(operation_holder, session):
    for op in operation_holder:
        session.run(op)
    

In [7]:

from collections import deque
# %matplotlib inline
# %matplotlib notebook
# # %load_ext autoreload
# # %autoreload 2


# fig = plt.figure(figsize=(12, 6))
# ax = fig.add_subplot(111)
# fig.show()
# fig.canvas.draw()

env = Environment(4, 4, "moment")
env.reset()

H = [32] # hidden neurons
D = env.state.size # input (state of the environment)
learning_rate = 1e-2
gamma = 0.99 # discount factor
epochs = 50000
buffer_size = 5000
max_frames = 150
action_space = 8

contin = 1

if not contin:
    eps = 0.3
    tf.reset_default_graph()
    agent = Agent(D, H, action_space, learning_rate, "agent")
    target = Agent(D, H, action_space, learning_rate, "target")
    
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    buffer = deque()
    
    # the first half of the list are the variables of the agent.
    # The last half the variables of the target
    variables = tf.trainable_variables() 
    operation_holder = prepare_update_target(variables)
    scores = deque()

last_ep = 0


#https://github.com/awjuliani/DeepRL-Agents/blob/master/Q-Network.ipynb
train_count = 0
loss = 0

for ep in range(epochs):
    
    if eps > 0.1 and len(buffer) >= buffer_size:
        eps *= 0.999
    
    if (ep + 1) % 100 == 0 and len(buffer) >= buffer_size:
        print(np.mean(list(scores)[-1000:]), "train_count", train_count,
              "loss", loss, "eps", eps)
        
    s = env.reset()
    s = [s]
    for c in range(max_frames):
        
        Q = sess.run(agent.predict_Q, {agent.input_s: s})
        
        if np.random.rand(1) < eps:
            rand = True
            a = env.sample_action()
        else:
            rand = False
            a = np.argmax(Q)
        
        s_new, r, done = env.step(a)
        scores.append(r)
        
        if rand == True:
            buffer.append([s, a, r, s_new, done])
        if rand == False and c % 50 == 0:
            buffer.append([s, a, r, s_new, done])
        
        if len(buffer) > buffer_size:
            buffer.popleft()
            scores.popleft()
        s = [s_new]
                
        if done:
            
            if len(buffer) >= buffer_size:
                batch = np.vstack(buffer)
                batch = batch[np.random.randint(0, buffer_size, size=500)]

                s = np.vstack(batch[:, 0])
                s_new = np.vstack(batch[:, 3])
                r = batch[:, 2]
                a = batch[:, 1]
                done_ = np.array(batch[:, 4], dtype=bool)
                Q = sess.run(agent.predict_Q, {agent.input_s: s})

                # Double Q-learning primary network chooses an action:
                Q_i = np.argmax(sess.run(agent.predict_Q, {agent.input_s: s_new}), 1)
                
                # Target network produces the Q-value of the chosen action
                Q_new = sess.run(target.predict_Q, {target.input_s: s_new})
                max_Q_new =Q_new[np.arange(Q_i.size)[:, None], Q_i[:, None]][:, -1]
                
                target_Q = r + gamma * max_Q_new
                target_Q[done_] = r[done_]
  
                train_count, loss, _ = sess.run([agent.train_count, agent.loss, agent.train], 
                                                feed_dict={agent.input_s: s, 
                                                           agent.executed_actions: a, 
                                                           agent.next_Q_r: target_Q})
                if c % 3 == 0:
                    # update target network
                    update_target(operation_holder, sess)

            break
            
            



0.667360347494 train_count 15488 loss 0.477816 eps 0.09990628614486748
0.443707837601 train_count 15587 loss 1.38988 eps 0.09990628614486748
0.524500981561 train_count 15686 loss 0.593845 eps 0.09990628614486748
0.359304577248 train_count 15785 loss 4.05721 eps 0.09990628614486748
0.374203011064 train_count 15884 loss 0.752228 eps 0.09990628614486748
0.580970897654 train_count 15984 loss 0.668351 eps 0.09990628614486748
0.876998766431 train_count 16084 loss 5.69235 eps 0.09990628614486748
0.0465974575166 train_count 16178 loss 0.603223 eps 0.09990628614486748
0.0747488582335 train_count 16276 loss 0.859723 eps 0.09990628614486748
0.0416824847548 train_count 16373 loss 1.52317 eps 0.09990628614486748
0.215243585896 train_count 16471 loss 4.26813 eps 0.09990628614486748
0.539468365938 train_count 16570 loss 2.51492 eps 0.09990628614486748
0.0675016616232 train_count 16667 loss 0.938396 eps 0.09990628614486748
0.211260882325 train_count 16767 loss 1.46387 eps 0.09990628614486748
0.1016234

KeyboardInterrupt: 

In [406]:
len(buffer)

32749

In [161]:
agent.saver.save(sess, "/home/ritchie46/Downloads/model_anastruct/model_bridge_4_2_moment.ckpt")


'/home/ritchie46/Downloads/model_anastruct/model_bridge_4_2_moment.ckpt'

In [19]:
agent.saver.save(sess, r"G:\bridge_builder\model_bridge_4_2_axial\model.ckpt")

'G:\\bridge_builder\\model_bridge_4_2_axial\\model.ckpt'

In [10]:
env = Environment(4, 4)
s = env.reset()


"""    
    → 0
    ↗ 1
    ↑ 2
    ↖ 3
    ← 4
    ↙ 5
    ↓ 6
    ↘ 7
"""

bot = agent

total_r = 0
j = 0
for a in [1, 0, 7, 7, 1, 0, 1]:
    j += 1

    print("\n", env.state)
    
    
    a_dst = sess.run(bot.predict_Q, {bot.input_s: [s]})
    a = np.argmax(a_dst)
    #a = np.random.choice(np.arange(8), p=a_dst[0])

    s, r, d = env.step(a)

    print(a_dst)
    print("action", a)
    print(r, d)
    total_r += r
    
#     if d == True:
#         j = 0
#         print(env.state)
#         break
#         #env.reset()
    
print("\r", total_r, env.structure())


 [[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 1.  0.  0.  0.]]
[[-0.12808736 -0.09334762 -0.10103387 -0.23614739 -0.23016927 -0.19282101
  -0.19304731 -0.2984907 ]]
action 1
-0.1 False

 [[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  2.  0.  0.]
 [ 1.  0.  0.  0.]]
[[-0.10368284 -0.11574875 -0.12774605 -0.12350658 -0.1642811  -0.20029946
  -0.16407546 -0.08707853]]
action 7
-0.1 False

 [[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  2.  0.  0.]
 [ 1.  0.  3.  0.]]
[[ 2.46880364 -0.09685077 -0.15725884 -0.14345071 -0.17737159 -0.20034733
  -0.20910886 -0.16599104]]
action 0
2.51404717866 True

 [[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  2.  0.  0.]
 [ 1.  0.  3.  4.]]
[[-0.11165825 -0.13312925 -0.15180516 -0.14280865 -0.20325702 -0.18633628
  -0.0402295   0.09051565]]
action 7
2.41404717866 True

 [[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  2.  0.  0.]
 [ 1.  0.  3.  4.]]
[[-0.11165825 -0.13312925 -0.15180516 -0.14280865 -0.20325702 -0.18633628
  -0.0402295   0.09051565]]

In [9]:
env = Environment(4, 4)
env.reset()
env.step(1)
env.step(0)
env.step(7)

(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  0., -1.,
         0.,  0.,  1.]), 10.140799116945235, True)